In [8]:
import os
import json
from pathlib import Path
from dotenv import load_dotenv
import gradio as gr
import ollama
from openai import OpenAI


In [ ]:
load_dotenv()

HF_TOKEN = os.getenv("HF_TOKEN", "")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY", "")

OPENAI_MODEL = "gpt-4o-mini"
LLAMA_MODEL = "llama3.1:8b"

HF_FREE_CHAT_MODEL = "HuggingFaceTB/SmolLM3-3B:hf-inference"
#HF_FREE_CHAT_MODEL="katanemo/Arch-Router-1.5B"
HF_ROUTER_BASE_URL = "https://router.huggingface.co/v1"

openai = OpenAI()
hf_client = OpenAI(base_url=HF_ROUTER_BASE_URL, api_key=HF_TOKEN)


In [10]:
def notebook_code_extractor(path: str) -> str:
    nb = json.loads(Path(path).read_text(encoding="utf-8"))
    parts = []
    for cell in nb.get("cells", []):
        if cell.get("cell_type") == "code":
            parts.append("".join(cell.get("source", [])))
    return "\n\n".join(parts).strip()


In [11]:
system_message_comments = (
    "You are a senior developer. Improve the code documentation by adding docstrings and short, useful comments. "
    "Keep it natural and practical. Do not over-comment obvious lines. "
    "Reply with code only."
)

system_message_summary = (
    "You are a senior developer. Summarize the code clearly: what it does, overall flow, inputs/outputs, and key points. "
    "Do not show the code. Do not use Markdown. Reply with plain text only."
)

def user_prompt_for(code: str) -> str:
    return "Add docstrings and helpful comments. Reply with code only.\n\n" + code

def user_prompt_for_summary(code: str) -> str:
    return "Summarize this code.\n\n" + code

def messages_for(code: str):
    return [
        {"role": "system", "content": system_message_comments},
        {"role": "user", "content": user_prompt_for(code)},
    ]

def messages_for_summary(code: str):
    return [
        {"role": "system", "content": system_message_summary},
        {"role": "user", "content": user_prompt_for_summary(code)},
    ]


In [12]:
def call_llama_local(code: str):
    r1 = ollama.chat(model=LLAMA_MODEL, messages=messages_for(code))
    r2 = ollama.chat(model=LLAMA_MODEL, messages=messages_for_summary(code))
    return r1["message"]["content"], r2["message"]["content"]

def call_gpt(code: str):
    c1 = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(code))
    c2 = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for_summary(code))
    return c1.choices[0].message.content, c2.choices[0].message.content

def call_hf_free(code: str):
    if not HF_TOKEN:
        raise RuntimeError("HF_TOKEN is not set in your environment.")
    c1 = hf_client.chat.completions.create(
        model=HF_FREE_CHAT_MODEL,
        messages=messages_for(code),
        max_tokens=1000,
    )
    c2 = hf_client.chat.completions.create(
        model=HF_FREE_CHAT_MODEL,
        messages=messages_for_summary(code),
        max_tokens=1000,
    )
    return c1.choices[0].message.content, c2.choices[0].message.content


In [13]:
import traceback

def document_uploaded_notebook(file_obj, model: str):
    try:
        if file_obj is None:
            return "ERROR: Please upload a .ipynb file.", "", ""

        path = file_obj.name
        if not path.lower().endswith(".ipynb"):
            return "ERROR: The uploaded file is not a .ipynb notebook.", "", ""

        code = notebook_code_extractor(path)
        if not code.strip():
            return "ERROR: No code cells found in the notebook.", "", ""

        m = (model or "").strip().lower()
        if m.startswith("llama"):
            commented, summary = call_llama_local(code)
        elif m.startswith("gpt"):
            commented, summary = call_gpt(code)
        elif m.startswith("hf"):
            commented, summary = call_hf_free(code)
        else:
            return f"ERROR: Unsupported model: {model!r}", "", ""

        return code, commented, summary

    except Exception:
        return "ERROR:\n" + traceback.format_exc(), "", ""


In [ ]:
css = """
.comments {background-color: #00599C;}
.summary {background-color: #008B8B;}
"""

with gr.Blocks(css=css) as ui:
    gr.Markdown("### Notebook Documentation Tool\nUpload a notebook and generate docstrings/comments + a summary.")

    with gr.Row():
        nb_file = gr.File(label="Upload .ipynb", file_types=[".ipynb"])

    with gr.Row():
        model = gr.Dropdown(
            ["HF (free)", "Llama (local)", "GPT (API)"],
            label="Model",
            value="HF (free)",
        )

    with gr.Row():
        run = gr.Button("Generate documentation")

    with gr.Row():
        source_code = gr.Textbox(label="Extracted notebook code (read-only)", lines=14, interactive=False)

    with gr.Row():
        commented_code = gr.Textbox(label="Documented code", lines=14, elem_classes=["comments"])
        code_summary = gr.Textbox(label="Summary", lines=14, elem_classes=["summary"])

    run.click(
        document_uploaded_notebook,
        inputs=[nb_file, model],
        outputs=[source_code, commented_code, code_summary],
    )

ui.launch(inbrowser=True)


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "g:\joxemi_gimenez\000_enerlogix\udemy\projects\llm_engineering\.venv\Lib\site-packages\uvicorn\protocols\http\httptools_impl.py", line 409, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "g:\joxemi_gimenez\000_enerlogix\udemy\projects\llm_engineering\.venv\Lib\site-packages\uvicorn\middleware\proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "g:\joxemi_gimenez\000_enerlogix\udemy\projects\llm_engineering\.venv\Lib\site-packages\fastapi\applications.py", line 1134, in __call__
    await super().__call__(scope, receive, send)
  File "g:\joxemi_gimenez\000_enerlogix\udemy\projects\llm_engineering\.venv\Lib\site-packages\starlette\applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  F